In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import datetime

## fetch all berlin weihnachtsmarkt links

In [2]:
url = 'https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/'
res = requests.get(url)
print("Successfully fetched data")

soup = BeautifulSoup(res.content, 'html.parser')
class_name = 'eintraege'
div_section = soup.find('div', class_=class_name)

links = div_section.find_all('a')
base_url = 'https://www.deutsche-weihnachtsmaerkte.de/'
wmlinks = [base_url + link['href'] for link in links]


Successfully fetched data


## getting dates and opening hours for each weihnachtsmarkt

In [3]:
dc = []
url = wmlinks[1]
print(url)
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')

https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-vor-dem-schloss-charlottenburg-1627.html


In [4]:
td_class_dates = 'oeffnungszeiten'
td_class_day = 'oeffnungszeiten-tag'
td_class_hours = 'oeffnungszeiten-zeit'
dc = {
    "name":[],
    "address":[],
    "url":[],
    "dates":[],
    "opening_hours":[]
}

for url in wmlinks:
    print(url)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')

    name = soup.find('h1', class_="underlined").text

    address = soup.find('div', class_="ort").find("p").text.replace("\n", " ")
    
    div_section = soup.find('div', class_=td_class_dates)
    dates = div_section.find('p')
    dates = dates.text if dates else None

    opening_hours = []
    
    rows = soup.select('table.oeffnungszeiten-tabelle tr')

    for row in rows:
        tag = row.select_one('.oeffnungszeiten-tag').get_text(strip=True)
        zeit = row.select_one('.oeffnungszeiten-zeit').get_text(strip=True)
        opening_hours.append((tag, zeit))

    dc["name"].append(name)
    dc["address"].append(address)
    dc["url"].append(url)
    dc["dates"].append(dates)
    dc["opening_hours"].append(opening_hours)
    time.sleep(1)
    
    

https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-evangelisches-johannesstift-berlin-1585.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-vor-dem-schloss-charlottenburg-1627.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtszeit-rotes-rathaus-berlin-352.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/lichtenberger-lichtermarkt-berlin-378.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/advents-oekomarkt-gruene-liga-berlin-379.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/lucia-weihnachtsmarkt-kulturbrauerei-berlin-380.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/daenischer-adventsbasar-berlin-381.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/schwedischer-weihnachtsbasar-berlin-wilmersdorf-382.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsma

## data cleaning and creating full calendar

In [5]:
df = pd.DataFrame(dc)
df['start_date'] = df['dates'].apply(lambda x: '' if x is None else x.split('-')[0].strip())
df['start_date'] = df['start_date'].apply(lambda x: datetime.datetime.strptime(x, '%d.%m.%Y') if x else None)
df['end_date'] = df['dates'].apply(lambda x: '' if x is None else x.split('-')[1].strip())
df['end_date'] = df['end_date'].apply(lambda x: datetime.datetime.strptime(x, '%d.%m.%Y') if x else None)
df['days'] = df['opening_hours'].apply(lambda x: [d[0] for d in x])

german_days = ["MO", "DI", "MI", "DO", "FR", "SA", "SO"]
german_day_mapper = {
    german_days[i]:i for i in range(len(german_days))
}

df['day_indexes'] = df['days'].apply(lambda x: set([german_day_mapper[d] for d in x]))
df

,name,address,url,dates,opening_hours,start_date,end_date,days,day_indexes
0,Weihnachtsmarkt Evangelisches Johannesstift Be...,Schönwalder Allee 26 13587 Berlin,https://www.deutsche-weihnachtsmaerkte.de/weih...,03.12.2023 - 03.12.2023,"[(SO, 12:00 - 18:00 Uhr)]",2023-12-03,2023-12-03,[SO],{6}
1,Weihnachtsmarkt vor dem Schloss Charlottenburg,zw. Luisen- & Klausenerplatz Spandauer Damm 14...,https://www.deutsche-weihnachtsmaerkte.de/weih...,27.11.2023 - 31.12.2023,"[(MO, 12:00 - 22:00 Uhr), (DI, 12:00 - 22:00 U...",2023-11-27,2023-12-31,"[MO, DI, MI, DO, FR, SA, SO]","{0, 1, 2, 3, 4, 5, 6}"
2,Weihnachtszeit Rotes Rathaus Berlin,St. Marienkirche & Rotes Rathaus Karl-Liebknec...,https://www.deutsche-weihnachtsmaerkte.de/weih...,27.11.2023 - 07.01.2024,"[(MO, 12:00 - 22:00 Uhr), (DI, 12:00 - 22:00 U...",2023-11-27,2024-01-07,"[MO, DI, MI, DO, FR, SA, SO]","{0, 1, 2, 3, 4, 5, 6}"
3,Lichtenberger Lichtermarkt Berlin,Möllendorffstr. 6 10367 Berlin,https://www.deutsche-weihnachtsmaerkte.de/weih...,03.12.2023 - 03.12.2023,"[(SO, 13:00 - 18:00 Uhr)]",2023-12-03,2023-12-03,[SO],{6}
4,Advents-Ökomarkt Grüne Liga Berlin,Kollwitzplatz 10405 Berlin,https://www.deutsche-weihnachtsmaerkte.de/weih...,03.12.2023 - 24.12.2023,"[(SO, 12:00 - 19:00 Uhr)]",2023-12-03,2023-12-24,[SO],{6}
5,Lucia Weihnachtsmarkt Kulturbrauerei Berlin,KulturBrauerei Schönhauser Allee 36 10435 Berlin,https://www.deutsche-weihnachtsmaerkte.de/weih...,27.11.2023 - 22.12.2023,"[(MO, 15:00 - 22:00 Uhr), (DI, 15:00 - 22:00 U...",2023-11-27,2023-12-22,"[MO, DI, MI, DO, FR, SA, SO]","{0, 1, 2, 3, 4, 5, 6}"
6,Dänischer Adventsbasar Berlin,Christianskirche Brienner Str. 12 10713 Berlin,https://www.deutsche-weihnachtsmaerkte.de/weih...,25.11.2023 - 25.11.2023,"[(SA, 12:00 - 22:00 Uhr)]",2023-11-25,2023-11-25,[SA],{5}
7,Schwedischer Weihnachtsbasar Berlin-Wilmersdorf,Schwedische Kirche Landhausstr. 26 - 28 10717 ...,https://www.deutsche-weihnachtsmaerkte.de/weih...,01.12.2023 - 03.12.2023,"[(FR, 16:00 - 20:00 Uhr), (SA, 12:00 - 19:00 U...",2023-12-01,2023-12-03,"[FR, SA, SO]","{4, 5, 6}"
8,Schöneberger Weihnachtsmarkt Berlin,Winterfeldtplatz 10781 Berlin,https://www.deutsche-weihnachtsmaerkte.de/weih...,None,[],NaT,NaT,[],{}
9,Lichtenrader Lichtermarkt Berlin,Dorfteich an der alten Dorfkirche Alt-Lichtenr...,https://www.deutsche-weihnachtsmaerkte.de/weih...,03.12.2023 - 03.12.2023,"[(SO, 13:00 - 19:00 Uhr)]",2023-12-03,2023-12-03,[SO],{6}


In [6]:
def find_available_christmas_markets(given_date, df):
    available_market_indexes = []
    for i in range(len(df)):
        if df.at[i, 'start_date'] <= given_date <= df.at[i, 'end_date'] and given_date.weekday() in df.at[i, 'day_indexes']:
            available_market_indexes.append(i)
    return available_market_indexes
            
find_available_christmas_markets(datetime.datetime.now(), df)

[21, 27, 42]

In [7]:
file_output = ""

start_date = datetime.datetime.now()
end_date = datetime.datetime(2023, 12, 31)
step = datetime.timedelta(days=1)

current_date = start_date
while current_date <= end_date:
    file_output+= "## " +current_date.strftime("%Y-%m-%d") + "\n"

    markets = find_available_christmas_markets(current_date, df)
    for i in markets:
        file_output += f"- [{df.at[i, 'name']}]({df.at[i, 'url']}), address: {df.at[i, 'address']}" + "\n"
    
    current_date += step

with open("calendar.md", "w") as f:
    f.write(file_output)